This notebook visualizes results from `SST_generation.py` and `SST_regression.py`.

In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot
matplotlib.rc_file('../rc_file')

In [ ]:
from paths import path_samoc, path_results, file_ex_ocn_ctrl
from timeseries import IterateOutputCESM
from ab_derivation_SST import DeriveSST as DS
from bc_analysis_fields import AnalyzeField as AF
from bd_analysis_indices import AnalyzeIndex as AI
from xr_regression import xr_quadtrend

## 1. generating full SST fields
### annual: from yearly TEMP_PD averaged data

In [ ]:
for run in ['ctrl', 'lpd']:
    plt.figure()
    da = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_{run}.nc', decode_times=False)
    da[0,:,:].plot(vmin=-1.8, vmax=30)

## 2. detrending/deseasonalizing

### Quadratic pointwise detrending
what detrending starting year to choose?

In [ ]:
da = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_ctrl.nc', decode_times=False)
# dt = xr.open_dataarray(f'{path_samoc}/SST/SST_GMST_sqdt_yrly_ctrl.nc', decode_times=False)
for lat in [400, 1700, 1800, 1900]:
    plt.figure()
    da.sel({'nlon':800, 'nlat':lat}).plot()
#     dt.sel({'nlon':800, 'nlat':lat}).plot()
    xr_quadtrend(da.sel({'nlon':800, 'nlat':lat})[150:]+1.5).plot()
    xr_quadtrend(da.sel({'nlon':800, 'nlat':lat})[100:]+1).plot()
    xr_quadtrend(da.sel({'nlon':800, 'nlat':lat})[40:]+.5).plot()
    xr_quadtrend(da.sel({'nlon':800, 'nlat':lat})).plot()


In [ ]:
da = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpd.nc', decode_times=False)

for lat in [50, 250, 300, 350]:
    plt.figure()
    da.sel({'nlon':0, 'nlat':lat}).plot()
#     dt.sel({'nlon':800, 'nlat':lat}).plot()
    xr_quadtrend(da.sel({'nlon':0, 'nlat':lat})[150:]+1.5).plot()
    xr_quadtrend(da.sel({'nlon':0, 'nlat':lat})[100:]+1).plot()
    xr_quadtrend(da.sel({'nlon':0, 'nlat':lat})[40:]+.5).plot()
    xr_quadtrend(da.sel({'nlon':0, 'nlat':lat})).plot()


### scaled GMSST detrending

In [ ]:
# %%time
# 1:32 min for both
for run in ['ctrl', 'lpd']:
    try:
        fn = f'{path_samoc}/SST/GMSST_yrly_{run}.nc'
        assert os.path.exists(fn)
        print(f'file exists: {fn}')
    except:
        DS().generate_yrly_global_mean_SST(run=run)

In [ ]:
for i, run in enumerate(['ctrl', 'lpd']):
    da = xr.open_dataarray(f'{path_samoc}/SST/GMSST_yrly_{run}.nc')
    plt.plot(da.time/365, da- i/2, lw=.5)
    if run=='ctrl':  x = da[40:]
    else:            x = da    
    pf = np.polynomial.polynomial.polyfit(x.time, x, 2)
    plt.plot(x.time/365, pf[2]*x.time**2 + pf[1]*x.time + pf[0] - i/2)
#     plt.axvline(40, c='grey', lw=.5)
    plt.ylabel('global mean SST [$^\circ$C]')
    plt.xlabel('time [years]')

## 3. derive raw SST indices

## 4. filter SST indices

In [ ]:
f, ax = plt.subplots(3, 1, figsize=(10,6))#, sharex=True)
for i, run in enumerate(['ctrl', 'lpd', 'had']):
    dt = [0, 200, 800][i]
    ax[2].set_xlabel('time [year]')
    for j, idx in enumerate(['AMO', 'SOM', 'TPI']):
        fn = f'{path_samoc}/SST/{idx}_{run}.nc'
        da = xr.open_dataarray(fn)
        if run=='had':
            label = 'two-factor'
            c = 'C2'
        else:
            label = 'scaled quadratic GMSST fit'
            c = 'C0'
        if i==0: label=None
        ax[j].plot(da[7:-7].time/365+dt, da[7:-7],         c=c, label=label)
        ax[j].plot(da[:7]  .time/365+dt, da[:7]  , ls=':', c=c)
        ax[j].plot(da[-7:] .time/365+dt, da[-7:] , ls=':', c=c)
            
        if run in ['ctrl', 'lpd']:
            for k, t in enumerate([ctrl_starts, lpd_starts][i]):
                tslice = (t, t+148)
                fns = f'{path_samoc}/SST/{idx}_{run}_{tslice[0]}_{tslice[1]}.nc'
                das = xr.open_dataarray(fns)
                if i==1 and k==0:  label='segment linear'
                else: label=None
                ax[j].plot(das[7:-7].time/365+dt, das[7:-7],         c='grey', lw=1, label=label)
                ax[j].plot(das[:7]  .time/365+dt, das[:7]  , ls=':', c='grey', lw=1)
                ax[j].plot(das[-7:] .time/365+dt, das[-7:] , ls=':', c='grey', lw=1)
            
            da = xr.open_dataarray(f'{path_samoc}/SST/{idx}_quadratic_pwdt_{run}.nc')
            
            if i==1: label = 'pointwise quadratic'
            ax[j].plot(da[7:-7].time/365+dt, da[7:-7],         c='C1', lw=1.5, label=label)
            ax[j].plot(da[:7]  .time/365+dt, da[:7]  , ls=':', c='C1', lw=1.5)
            ax[j].plot(da[-7:] .time/365+dt, da[-7:] , ls=':', c='C1', lw=1.5)
            
        ax[j].set_ylabel(idx)
        ax[j].axhline(0, c='k', lw=.5)
        if i>0 and j==1:
            ax[j].legend(ncol=4)
        ax[j].set_xticks(np.arange(0,1000,50))
ax[0].text(150, -.35, 'CTRL', ha='center')
ax[0].text(500, -.35, 'LPD' , ha='center')
ax[0].text(900, -.35, 'HAD' , ha='center')
ax[2].set_xlim((-20, 970))
plt.savefig(f'{path_results}/SST/SST_indices_different_dt')

spectra, mean map and correlation to obs, std of maps, correlations of maps with their mean

In [ ]:
from bb_analysis_timeseries import AnalyzeTimeSeries as ATS

In [ ]:
spec[2].T[1]

In [ ]:
for i, run in enumerate(['ctrl', 'lpd', 'had']):
    for j, idx in enumerate(['AMO', 'SOM', 'TPI']):
        fn = f'{path_samoc}/SST/{idx}_quadratic_pwdt_raw_{run}.nc'
        da = xr.open_dataarray(fn)
        spec = ATS(da).spectrum(filter_type='lowpass', filter_cutoff=13)
        plt.figure()
        plt.loglog(1/spec[1], spec[0])
        plt.loglog(1/spec[1], spec[2].T[1])

## Regression patterns

In [ ]:
from maps import regr_map

In [ ]:
for idx in ['AMO', 'SOM', 'TPI']:
    ds = xr.open_dataset(f'{path_samoc}/SST/{idx}_regr_had.nc')
    regr_map(ds=ds, index=idx, run='had')

### mean pattern for ctrl and lpd

In [ ]:
if run=='ctrl':   starts = np.arange(50, 151, 5)
elif run=='lpd':  starts = np.arange(154, 415, 10)

In [ ]:
for j, run in enumerate(['ctrl', 'lpd']):
    if run=='ctrl':   starts = np.arange(50, 151, 5)
    elif run=='lpd':  starts = np.arange(154, 415, 10)
        
    for k, t in enumerate(starts):
        tslice = (t, t+148)

### pattern correlation

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(8,3), sharey=True, constrained_layout=True)
for i, idx in enumerate(['AMO', 'SOM', 'TPI']):
    for j, run in enumerate(['ctrl', 'lpd']):  # , 
        fn = f'{path_samoc}/SST/{idx}_spatial_correlations_{run}.nc'
        da = xr.open_dataarray(fn)
        da.plot(label=idx, ax=ax[j])
        ax[j].set_xlabel('starting year of segment')
        ax[j].text(da.time[0], .7, run.upper())
        ax[j].tick_params()
        ax[j].axhline(0, c='k', lw=.3)

ax[0].legend(ncol=3, loc=8)
ax[0].set_ylabel('spatial correlation coefficient')
# plt.savefig(f'{path_results}/SST/spatial_correlation(t)_ctrl_lpd')

# Stationarity